<a href="https://colab.research.google.com/github/okana2ki/intro-to-DS/blob/main/DS13_0v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SSDSE-Cデータの分析

# ノートブックの保存／保存したものの再開

## **保存のしかた**

Colabで開いたら、まず、↑上部の**「ドライブにコピー」をクリック**して下さい。Googleドライブの [マイドライブ > Colab Notebooks] に保存され、その後加えた変更がそこに自動保存されるようになります。

ヘッダーに表示されるはずの「ドライブにコピー」が表示されてない場合は、画面上部右端の「v」アイコンをクリックして、ヘッダーの表示／非表示を切り替えて下さい。

スマホの場合は、左上の3本線を押して、「ファイル」-「ドライブにコピーを保存」のはず。

## **再開のしかた**

ヘッダー（上記参照）が表示されている状態で、左上の「ファイル」をクリック→「ノートブックを開く」をクリック→最近閲覧したノートブックの一覧が表示されるので、開きたいノートブック名をクリックするか、右端のいくつかのアイコンのうち目的に合ったものをクリック（または、左カラムの「Googleドライブ」をクリックするとドライブ中のノートブック一覧が表示されるのでそこから探す；一覧から探すときは、一覧上部の「↑」や「↓」をクリックすることでいろいろな条件で並べ替えできます）

# Gemini (Colab AI) の使い方

* Googleアカウントでログインした状態で、**PCで**このコラボ・ノートブックを開くとAIが手伝ってくれます。

次のようにAIに頼んでみよう。頼む手順：

0. コードセルを新たに追加したい場合は、ノートブック画面上部の「+コード」をクリックする(現在カーソルがあるセルの下に追加される)か、または、セルの間あたりにカーソルをもってくると表示される「+コード」をクリックする(クリックした場所に追加される)。

1. コードセルを作ったときの表示「コーディングを開始するか、AIで<u>生成</u>します。」中の「<u>生成</u>」をクリックすると（画面の一番下、中央のGeminiアイコンをクリックすることで対話を始めてもよい）、入力窓が現れるので、そこに以下の依頼をコピー(Ctrl-C)・アンド・ペースト(Ctrl-V)しよう←このショートカットキーはWindowsの場合（Ctrl-CはCtrlキーを押しながら同時にCキーを押すことを示す）；MacではCtrlの代わりにcommandキー）
2. 入力窓への入力が済んだら、Enterキーを押すか、入力窓の右下の「送信（紙飛行機）」アイコンを押す。

依頼の書き方（先週の小テストの例）：
> (1, 2, 3) という3個のデータの（不偏分散ではなく）標本分散を求めて下さい。

#ファイルの読み込み
CSVファイルをpandasのDataFrame形式で読み込む。



## ヘッダー行の指定

**ヘッダー行を行番号で指定する**必要がある。SSDSE-C-2024.csvの場合は、最初の2行がヘッダー（データは3行目から始まる）であるが、1行目は「項目コード」であるためAIに頼むときやプログラムを見たとき分かりにくく、2行目の「項目名」の方が分かりやすいので、下記のように2行目をヘッダーとして指定する。上から0行、1行と数えるので、**行番号1は2行目を示す**ことに注意。
```
header=1
```
なお、1, 2行目の両方をヘッダー行として指定することもできるが、プログラムが煩雑になるだけでメリットはあまり無さそうに思うので、このノートブックでは、2行目のみを指定する方式を採用する。唯一、メリットがあるとすると、「項目コード」は、12個の中分類（小計）に該当する列がどれで、それはどの列の小計であるかを、コードだけから機械的に判断できることだと思う。例：LB01で始まるコードは中分類の「穀類」に入る品目である。

## 文字コードの指定

日本語を含むファイルの場合は、encoding（文字コード）を適切に指定する必要がある。encodingを指定しないと、デフォルトのutf_8と見なされるが、それでエラーが出る場合は、shift_jisを指定する。
```
encoding="utf_8"  # こちらがデフォルト（指定しないとこちらだと解釈される）
encoding="shift_jis"
```



## Webサイト（SSDSEのサイト）上のファイルの読み込み

以下の演習では、`SSDSE-C-2025.csv` を読み込んで使用する。

2025年12月修正：
このウェブサイトは、自動ダウンロードスクリプトからのアクセスを防ぐためのセキュリティ対策を最近とるようになったため、それへの対策として、requestsライブラリを使ってウェブブラウザのように振る舞うUser-Agentヘッダーを付けてファイルをダウンロードし、その内容をpandas.read_csvに渡すようにコードを修正した。

In [ ]:
import pandas as pd
import requests
import io

url = "https://www.nstac.go.jp/files/SSDSE-C-2025.csv"

# User-Agentヘッダーを追加してブラウザからのアクセスを模倣
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # HTTPエラーがあれば例外を発生させる

    # shift_jisエンコーディングでコンテンツをデコードし、StringIOでファイルのように扱う
    df = pd.read_csv(io.StringIO(response.content.decode('shift_jis')), header=1)
    print(df)
except requests.exceptions.HTTPError as errh:
    print(f"HTTP Error: {errh}")
    print("ファイルへのアクセスが拒否されました。ウェブサイトが直接ダウンロードを防いでいる可能性があります。")
    print("解決策として、ファイルをブラウザで手動でダウンロードし、Colabにアップロードして読み込むことを試してください。")
except requests.exceptions.ConnectionError as errc:
    print(f"接続エラー: {errc}")
except requests.exceptions.Timeout as errt:
    print(f"タイムアウトエラー: {errt}")
except requests.exceptions.RequestException as err:
    print(f"予期せぬエラーが発生しました: {err}")
except UnicodeDecodeError as e:
    print(f"文字コードのデコードエラーが発生しました: {e}")
    print("encoding='shift_jis' が正しくない可能性があります。他のエンコーディングを試すか、ファイルの内容を確認してください。")


## AIへのデータ解説

**この後、AIにプログラム作成を依頼したときに、このデータを適切に解釈してもらえるように、↓下のセルで、コメント行の形で、このデータの形式と内容を（AIに対して）説明しておく。←これは重要です。これがあるのとないのとでは、適切なプログラムが生成される可能性が大きく変わるはず。** 試しに、下記のセルよりも上のセルでAIに依頼して比べてみて下さい（AIは依頼したセルよりも下は読み込まないので←最近、対話履歴をコンテクストとして利用しているようなので、必ずしもこのようには振る舞いません）。（もちろん、プログラム作成を依頼するときに、このような情報を付けて依頼することもできる。）

↓下のセル内の
説明は、[サイト](https://www.nstac.go.jp/use/literacy/ssdse/) に掲載されている[データ解説PDFファイル](https://www.nstac.go.jp/files/kaisetsu-C-2025.pdf) に基づいて書きました。（AIだけでなく）皆さんも、この解説PDFに目を通してからデータ分析するようにして下さい。

`header=1`と指定したため、変数`df`は、元のCSVファイルから、1行目を削除した内容になっていることに注意して下さい。

なお、Geminiとの対話窓の「＋」アイコンをクリックして、[データ解説PDFファイル](https://www.nstac.go.jp/files/kaisetsu-C-2025.pdf) をアップロードして、「データの形式や内容の解説は添付したファイルに記されているので、これを参照して、データを分析して」と頼むこともできるはず。

In [ ]:
# dfデータの解説
# データのレイアウト
# 1行目は項目名、それ以下の48行がデータ。データ行は、「全国」と「47都道府県庁所在市」に対応。
# 列は、最初の3列が地域情報を示す。1行目から順に「地域コード」「都道府県」「市」を示す。
# 4列目以降がデータで、4列目から順に「世帯人員」「食料（合計）」と続くが、5列目は、食品（合計）に対する1世帯当たり年間支出金額で、
# 6列目以降は、食品の品目ごとの1世帯当たり年間支出金額のデータである。
# なお、次の12個の列は、それぞれ、それ以降の列の小計になっている：「穀類」「魚介類」「肉類」「乳卵類」「野菜・海藻」「果物」「油脂・調味料」「菓子類」「調理食品」「飲料」「酒類」「外食」

# AIにいろいろな分析をお願いしてみよう！

エクセルで演習したような分析はもちろん、もっと面白い分析、もっと複雑な分析でも、言葉でお願いするだけで、手軽にできるよ。やりたい分析や欲しい結果と違う場合は、それを指摘して繰り返しお願いしてみよう。

AIに依頼すると、グラフ中に日本語表示するプログラムを作成することがあるため、グラフ中に日本語が表示できるように、あらかじめ、下記の2つのコードセルを実行しておきましょう。

In [ ]:
!pip install japanize_matplotlib

In [3]:
import japanize_matplotlib

下のコードセルで、AIに何か分析を頼んでみましょう。

## ソート

↓AIへの依頼と応答の例です。

In [ ]:
# prompt: ぎょうざの消費額が多い順にソートして下さい。
# dfにデータが読み込まれている前提のコードになっています。

# ぎょうざの列名を取得 (列名はデータによって異なる可能性があるので注意)
gyoza_column = "ぎょうざ"  # 列名を適切に設定してください
if gyoza_column not in df.columns:
  print(f"Error: Column '{gyoza_column}' not found in the DataFrame.")
  # 列が存在しない場合は、エラー処理や代替列名を使用するなど、適切な対応をしてください
else:
  # ぎょうざの消費額が多い順にソート
  sorted_df = df.sort_values(by=gyoza_column, ascending=False)
sorted_df

↑ソートできているようですが、「市」と「ぎょうざ」の列だけを取り出してソート結果を表示するためには、AIにどう頼めばよいでしょうか？

上のコードセルを下のコードセルにコピーして、下のコードセルで「コードの変換（変更）」を依頼してみて下さい。

下のコードセルは私が依頼してみた例です。

In [ ]:
# prompt: 「市」と「ぎょうざ」の列だけを取り出してソート結果を表示して下さい。

#「市」と「ぎょうざ」の列だけを取り出してソート結果を表示
# ぎょうざの列名を取得 (列名はデータによって異なる可能性があるので注意)
gyoza_column = "ぎょうざ"  # 列名を適切に設定してください

if gyoza_column not in df.columns:
    print(f"Error: Column '{gyoza_column}' not found in the DataFrame.")
else:
    # 「市」と「ぎょうざ」の列を選択
    selected_columns = df[["市", gyoza_column]]

    # ぎょうざの消費額が多い順にソート
    sorted_df = selected_columns.sort_values(by=gyoza_column, ascending=False)

    # 結果を表示
sorted_df

## マップ表示

AIに次のような依頼をしてみたら、どうなるでしょうか？地図上での表示はできるかな？

「こんにゃくの消費額が多い都道府県を地図上で表示したい。正確にはデータは都道府県庁所在市の消費額であるが、都道府県をマップ上で色分けすることで表示をしたいため、都道府県庁所在市のデータを都道府県を代表するデータだと考えることにする。全国平均より多い都道府県とそうでない都道府県を色分けして表示する。」

私が試したときには、AIは、日本の47都道府県のポリゴンデータGeoJSONを取得するところでつまずいてエラーが取れなくなったため、自分で都道府県のGeoJSONデータをダウンロードしてきて、それをAIに渡す方法を採ることにした。

ネット検索で見つけた[このサイト](https://japonyol.net/editor/article/47-prefectures-geojson.html)から、GeoJSONファイルをダウンロードして、それをAIに添付して渡して、マップ表示を依頼したらうまくいった。

参考までに、下記にこのときのやりとりの主要部を残しておきます。



---



## マップ表示TaskにおけるAIからの出力（主要な部分を抜粋）
それでは、まず日本の都道府県のGeoJSONデータを取得しましょう。地図の描画には、各都道府県の地理情報が必要になります。

以下のコードセルでは、日本の都道府県のGeoJSONデータをGitHubのリポジトリから取得し、`folium`で利用するために準備します。

GeoJSONデータは`prefectures_geojson`という変数に格納されます。

### 日本の都道府県のGeoJSONデータを取得する

### Subtask:
日本地図の描画に必要な都道府県のGeoJSONデータをインターネットから取得します。foliumなどのライブラリで使用するために適切な形式であることを確認します。


**Reasoning**:
First, I'll import the necessary libraries (`requests` and `json`) to handle web requests and JSON parsing. Then, I'll define the URL for the GeoJSON data and fetch it using `requests.get()`. After fetching, I'll parse the response content into a Python dictionary and store it in `prefectures_geojson`. Finally, I'll print the first feature to confirm the data was loaded correctly.



（以下、エラーが出続けた長いやりとりは省略）

**エラーが取れないので、自分でGeoJSONデータをダウンロードしてきて、それをAIに渡す方針に変更↓**

### GeoJSONデータを取得する（ファイルから読み込み）

### Subtask:
ユーザーから提供されたGeoJSONファイル `/content/prefectures.geojson` を読み込み、foliumなどのライブラリで利用するために準備します。


**Reasoning**:
I need to load the GeoJSON data from the local file `/content/prefectures.geojson` using the `json` library. I will open the file with `utf-8` encoding and then parse it into the `prefectures_geojson` variable. Finally, I will print the first feature to confirm the data is loaded correctly.



**↓このコードは、prefectures.geojsonをコラボからアクセスできる場所'/content/prefectures.geojson'に置いてあることが前提**です。prefectures.geojsonのダウンロードは、[ここ](https://japonyol.net/editor/article/47-prefectures-geojson.html)からできます。そのファイルを'/content/prefectures.geojson'に置くための操作は、Geminiに尋ねてみて下さい。

In [ ]:
import json

# GeoJSONファイルを読み込む
with open('/content/prefectures.geojson', 'r', encoding='utf-8') as f:
    prefectures_geojson = json.load(f)

# 取得したGeoJSONデータの最初のいくつかのフィーチャを表示して、データが正しく読み込まれたことを確認
print(prefectures_geojson['features'][0])

### こんにゃくの消費額データを準備する

### Subtask:
DataFrame `df` から「こんにゃく」の消費額データを抽出し、全国平均の消費額を算出します。各都道府県庁所在市の消費額が全国平均より高いか低いかを判断する新しい列を作成します。この際、「全国」の行は全国平均の算出に利用し、都道府県ごとのデータからは除外します。


**Reasoning**:
First, I need to select the specified columns ('都道府県', '市', 'こんにゃく') from the `df` DataFrame to create `konnyaku_data`. Then I will extract the national average consumption for 'こんにゃく' from the '全国' row. After that, the '全国' row will be dropped from the `konnyaku_data` to ensure only prefectural data remains for comparison. Finally, a new boolean column '全国平均より高い' will be created to indicate if each prefecture's consumption is above the national average, and the first few rows of the prepared data will be displayed.



In [ ]:
konnyaku_data = df[['都道府県', '市', 'こんにゃく']].copy()

national_average_row = konnyaku_data[konnyaku_data['都道府県'] == '全国']
national_average_konnyaku = national_average_row['こんにゃく'].iloc[0]

konnyaku_data = konnyaku_data[konnyaku_data['都道府県'] != '全国']

konnyaku_data['全国平均より高い'] = konnyaku_data['こんにゃく'] > national_average_konnyaku

print(konnyaku_data.head())

### Choroplethマップを作成する

### Subtask:
取得したGeoJSONデータと準備したこんにゃく消費額データを結合し、foliumライブラリを使用してChoroplethマップを作成します。全国平均より消費額が高い都道府県と低い都道府県で色分けし、マップ上に都道府県名と消費額を表示するツールチップを設定します。


その後、消費額に応じたカラーで表示されるように変更したコードが下記です↓

In [ ]:
import folium
import pandas as pd

# 日本の中心座標
center_lat = 37.0
center_lon = 138.0
m = folium.Map(location=[center_lat, center_lon], zoom_start=5)

# konnyaku_dataを都道府県名でインデックス化しておくと、GeoJSONデータとの結合がしやすくなります。
konnyaku_indexed = konnyaku_data.set_index('都道府県')

# GeoJSONデータにこんにゃく消費データを追加
# ここでは、数値データのみをGeoJSONのpropertiesに直接追加します。
# Choroplethレイヤーがこの値を使って色付けするため、'全国平均より高い'のロジックは不要になります。
for feature in prefectures_geojson['features']:
    prefecture_name = feature['properties']['name']
    if prefecture_name in konnyaku_indexed.index:
        feature['properties']['こんにゃく'] = int(konnyaku_indexed.loc[prefecture_name, 'こんにゃく'])
    else:
        feature['properties']['こんにゃく'] = None  # データがない場合はNoneを設定

# Choroplethレイヤーを作成し、こんにゃく消費額に基づいて色を割り当てます
# threshold_scaleを省略することで、foliumが自動的に分位点に基づく階級を生成します
folium.Choropleth(
    geo_data=prefectures_geojson,
    name='こんにゃく消費額',
    data=konnyaku_data, # 元のデータフレームを使用
    columns=['都道府県', 'こんにゃく'], # 都道府県名と消費額の列を指定
    key_on='feature.properties.name', # GeoJSONの都道府県名プロパティに結合
    fill_color='YlOrRd', # 黄色から赤へのグラデーション
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='こんにゃく消費額 (円)',
    highlight=True,
    line_color='black',
).add_to(m)

# ツールチップのカスタマイズ
# Choroplethレイヤーの後にGeoJsonレイヤーを追加して、より詳細なツールチップを設定します
# ただし、これまでのGeoJsonTooltipのロジックを再利用します
folium.GeoJson(
    prefectures_geojson,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name', 'こんにゃく'],
        aliases=['都道府県名', 'こんにゃく消費額'],
        localize=True,
        labels=True,
        sticky=False
    )
).add_to(m)

# マップを表示
display(m)

## 宮崎に関する面白いデータ分析結果（冬休みの宿題）

これもAIに上手に頼むといろいろな面白い分析結果を見せてくれるはず。いろいろ楽しんで下さい。

🎅メリークリスマス！＆🎍よいお年を！



---



# クイズ出題のためのシステムインストラクション

ここで、クイズを出してくれるようインストラクションを書いたコードセルを挿入しておきましょう。クイズがじゃまだったら、↓下のコードセルを消すか、下方に移動して下さい。下方に移動するとそこより下のAIへの依頼のときだけ、このインストラクションが参照されます。←以前はこの挙動でしたが、2025年秋以降、過去の対話履歴が参照されるようになり、昔依頼したいクイズ出題の影響が残るようになりました。

セルの消去は、セルをクリックして右上に現れるアイコンの中から「ゴミ箱」をクリック。下方への移動は、現れたアイコンの中から「↓」をクリック。

In [ ]:
# system instruction:
# ユーザはプログラミング初心者です。
# あなたの役割は、ユーザのコーディング依頼に応じる際、ユーザがプログラミングを習得できるようにすることです。
# ユーザのコーディング依頼に対してコードを生成する場合、コードの生成に加えて、
# 「Pythonコードの書き方に関してユーザは何が分かってないかを自覚させるクイズ」を日本語で出題して下さい。
# さらに、クイズの解答に応じてユーザに日本語でフィードバックして下さい。

以下で、いろいろなデータ分析を自由に試してみて下さい。